# FID Score Pytorch

Following the tutorial:

- https://machinelearningmastery.com/how-to-implement-the-frechet-inception-distance-fid-from-scratch/


## Native Implementation

In [77]:
#%matplotlib inline
import os
import random
import torch
import numpy as np
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils

from PIL import Image

from scipy import linalg

from numpy.random import randint

In [94]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=True)
model.eval()
""

Using cache found in C:\Users\willf/.cache\torch\hub\pytorch_vision_v0.10.0


''

In [88]:
def calculate_frechet_distance(mu1, sigma1, mu2, sigma2, eps=1e-6):
    """Numpy implementation of the Frechet Distance.
    The Frechet distance between two multivariate Gaussians X_1 ~ N(mu_1, C_1)
    and X_2 ~ N(mu_2, C_2) is
            d^2 = ||mu_1 - mu_2||^2 + Tr(C_1 + C_2 - 2*sqrt(C_1*C_2)).
    """

    mu1 = np.atleast_1d(mu1)
    mu2 = np.atleast_1d(mu2)

    sigma1 = np.atleast_2d(sigma1)
    sigma2 = np.atleast_2d(sigma2)

    assert mu1.shape == mu2.shape, \
        'Training and test mean vectors have different lengths'
    assert sigma1.shape == sigma2.shape, \
        'Training and test covariances have different dimensions'

    diff = mu1 - mu2

    covmean, _ = linalg.sqrtm(sigma1.dot(sigma2), disp=False)
    if not np.isfinite(covmean).all():
        msg = ('fid calculation produces singular product; '
               'adding %s to diagonal of cov estimates') % eps
        print(msg)
        offset = np.eye(sigma1.shape[0]) * eps
        covmean = linalg.sqrtm((sigma1 + offset).dot(sigma2 + offset))

    
    if np.iscomplexobj(covmean):
        if not np.allclose(np.diagonal(covmean).imag, 0, atol=1e-3):
            m = np.max(np.abs(covmean.imag))
            raise ValueError('Imaginary component {}'.format(m))
        covmean = covmean.real

    tr_covmean = np.trace(covmean)

    return (diff.dot(diff) + np.trace(sigma1) +
            np.trace(sigma2) - 2 * tr_covmean)

In [95]:
# calculate frechet inception distance
def calculate_fid(model, images1, images2):
	# calculate activations
	with torch.no_grad():
		act1 = model(images1).cpu().detach().numpy()
		act2 = model(images2).cpu().detach().numpy()
	# calculate mean and covariance statistics
	mu1, sigma1 = np.mean(act1, axis=0), np.cov(act1, rowvar=False)
	mu2, sigma2 = np.mean(act2, axis=0), np.cov(act2, rowvar=False)
	
	return calculate_frechet_distance(mu1, sigma1, mu2, sigma2)

In [100]:
preprocess = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [102]:
input_image = Image.open('../test_images/flower1.jpg')
input_tensor = preprocess(input_image)
input_batch_1 = input_tensor.unsqueeze(0)

input_image = Image.open('../test_images/flower2.jpg')
input_tensor = preprocess(input_image)
input_batch_2 = input_tensor.unsqueeze(0)

In [105]:
# fid between images1 and images1
fid = calculate_fid(model, input_batch_1, input_batch_1)
print('FID (same): %.3f' % fid)
# fid between images1 and images2
fid = calculate_fid(model, input_batch_1, input_batch_2)
print('FID (different): %.3f' % fid)

FID (same): 0.000
FID (different): 889.460


## Using Pytorch-FID Library

In [7]:
# %pip install pytorch_fid

In [8]:
from pytorch_fid import fid_score, inception

In [ ]:
device = torch.device('cpu')
dim = 2048
paths = []
for name in dir_names:
    path = tmp_path / name
    path.mkdir()
    paths.append(str(path))

fid_value = fid_score.calculate_fid_given_paths(paths,
    batch_size=dim,
    device=device,
    dims=dim,
    num_workers=0)

## Using Ignite Library

In [6]:
# %pip install pytorch-ignite

In [3]:
from collections import OrderedDict

import torch
import scipy
from torch import nn, optim

from ignite.engine import *
from ignite.handlers import *
from ignite.metrics import *
from ignite.utils import *
from ignite.contrib.metrics.regression import *
from ignite.contrib.metrics import *

# create default evaluator for doctests

def eval_step(engine, batch):
    return batch

default_evaluator = Engine(eval_step)

# create default optimizer for doctests

param_tensor = torch.zeros([1], requires_grad=True)
default_optimizer = torch.optim.SGD([param_tensor], lr=0.1)

# create default trainer for doctests
# as handlers could be attached to the trainer,
# each test must define his own trainer using `.. testsetup:`

def get_default_trainer():

    def train_step(engine, batch):
        return batch

    return Engine(train_step)

# create default model for doctests

default_model = nn.Sequential(OrderedDict([
    ('base', nn.Linear(4, 2)),
    ('fc', nn.Linear(2, 1))
]))

manual_seed(666)

In [4]:
metric = FID(num_features=1, feature_extractor=default_model)
metric.attach(default_evaluator, "fid")
y_true = torch.ones(10, 4)
y_pred = torch.ones(10, 4)
state = default_evaluator.run([[y_pred, y_true]])
print(state.metrics["fid"])

0.0


c:\Users\willf\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\linalg\_matfuncs_sqrtm.py:189: RuntimeWarning: invalid value encountered in double_scalars
  arg2 = norm(X.dot(X) - A, 'fro')**2 / norm(A, 'fro')
